In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

In [2]:
# message class
@dataclass
class Message:
    content: str

### host for distributed runtime

In [4]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [5]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [6]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [8]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [9]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [10]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [11]:
display(Markdown(response.content))

## Pros of AutoGen:
AutoGen offers several compelling advantages for AI agent development:

*   **Multi-Agent Collaboration:** AutoGen excels at facilitating collaboration between multiple specialized agents, allowing them to work together to solve complex problems, generate code, and perform tasks autonomously. This is a core strength.
*   **LLM Optimization:** AutoGen helps optimize the use of Large Language Models (LLMs) through effective agent orchestration.
*   **Tool Integration:** It supports integration with various tools, including code executors and function callers, enabling agents to perform complex tasks.
*   **Compatibility:** AutoGen is compatible with different LLMs and toolchains, providing flexibility in agent design.
*   **Human Oversight:** It supports systems that involve human oversight in the agent workflow.
*   **Efficiency, Scalability, and Problem-Solving Depth:** By fostering collaboration, AutoGen improves these aspects of AI-powered solutions.

In summary, if your project requires multiple agents working together, complex task execution, or human oversight, AutoGen is a strong choice due to its capabilities in multi-agent collaboration, LLM optimization, and tool integration.

TERMINATE


## Cons of AutoGen:
Here are some cons of using AutoGen for an AI Agent project:

*   **Steeper Learning Curve:** AutoGen can have a steeper learning curve compared to some other frameworks or low-code platforms. It requires some level of coding expertise, potentially limiting accessibility for non-technical users.
*   **Complexity:** While its flexibility is a strength, it can also lead to complexity, especially when dealing with intricate agent interactions or unstructured workflows. Setting up the desired agent conversation patterns might require more effort.
*   **Requires more initial effort:** Compared to more streamlined frameworks, AutoGen often demands more initial effort to define roles and workflows.
*   **Limited out-of-the-box functionality:** AutoGen is more of a foundational framework, requiring more manual configuration and integration of tools compared to some more specialized agent platforms that offer more pre-built functionalities.

TERMINATE




## Decision:

Decision: It depends on the project requirements and team expertise.

Rationale: AutoGen is a powerful framework for multi-agent collaboration, LLM optimization, and tool integration. However, it has a steeper learning curve, can be complex to configure, requires more initial effort, and has limited out-of-the-box functionality. Therefore, if the project requires complex agent interactions, benefits from human oversight, and the team has the necessary coding expertise, AutoGen could be a strong choice. But, if the project is simple, requires a quick setup, or the team lacks the coding skills, other frameworks might be more suitable.


In [12]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [13]:
await host.stop()